In [57]:
# load dataset
import json

path = './hotpot_train_v1.1.json'

with open(path) as f: 
    data = json.load(f) 

In [71]:
# calculate number of duplicates
duplicates = 0
for sample in data:
    supporting = [sample['supporting_facts'][0][0], sample['supporting_facts'][1][0]]
    passages = {}
    for passage in sample['context']:
        passages[passage[0]] = ' '.join(passage[1])

    supporting = [passages[supporting[0]], passages[supporting[1]]]

    if supporting[0] == supporting[1]:
        duplicates += 1
        if duplicates == 1:
            print(supporting[0], '\n')
            print(supporting[1])
        
print(duplicates)

Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.  Goertz is known for her satirical songs based on various pop culture topics.  Her videos are posted on YouTube under the name of Cossbysweater.  Subjects of her songs have included the film "The Room", the character Milhouse from the television show "The Simpsons", and the game Dungeons & Dragons.  Her style has been compared to that of Bo Burnham.  In December 2015, Goertz released a concept album based on the Adult Swim series "Rick and Morty", "Sad Dance Songs", with the album's cover emulating the animation and logo of the series.  The album was made possible through Kickstarter.  She is co-host of Everything's Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. 

Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.  Goertz is known for her satirical songs based on various pop culture topics.  Her videos are posted on YouTube under the name of Cossbysweater.  Subject

In [70]:
# examine
sample = data[11]
question = sample['question']
a = sample['answer']
supporting = [sample['supporting_facts'][0][0], sample['supporting_facts'][1][0]]

passages = {}
for passage in sample['context']:
    passages[passage[0]] = ' '.join(passage[1])
    
supporting = [passages[supporting[0]], passages[supporting[1]]]

print(question)
print(supporting[0], '\n')
print(supporting[1])
print(a)
print(sample['type'])

sample = [question] + supporting + [a]
# print(sample)

Gunmen from Laredo starred which narrator of "Frontier"?
Gunmen from Laredo is a 1959 American western film produced and directed by Wallace MacDonald, which stars Robert Knapp, Maureen Hingert, and Walter Coy. 

Walter Darwin Coy (January 31, 1909 – December 11, 1974) was an American stage, radio, film, and, principally, television actor, originally from Great Falls, Montana.  He was best known for narrating the NBC western anthology series, "Frontier", which aired early Sunday evenings in the 1955–1956 season.
Walter Darwin Coy
bridge


In [37]:
# preprocess
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenised_sample = []
for c in sample:
    words = tokenizer.tokenize(c)
    tokenised_sample.append(words)
print(tokenised_sample)
q, p1, p2, a = tokenised_sample

[['where', 'did', 'the', 'form', 'of', 'music', 'played', 'by', 'die', 'rhone', '##r', 'sa', '##uw', '##ant', '##z', '##t', 'originate', '?'], ['die', 'rhone', '##r', 'sa', '##uw', '##ant', '##z', '##t', 'are', 'a', 'ski', '##ffle', '-', 'blues', '##band', 'from', 'e', '##iche', '##nz', '##ell', '-', 'lu', '##tter', 'in', 'hesse', '##n', ',', 'germany', '.', 'the', 'line', '-', 'up', 'consists', 'of', 'martin', 'cab', '##a', ',', 'christoph', 'gunther', 'and', 'christoph', 'lei', '##pol', '##d', 'playing', 'ski', '##ffle', '-', 'blues', 'with', 'lyrics', 'based', 'on', 'r', '##hon', 'mountains', 'dialect', 'and', 'other', 'hess', '##ian', 'dialects', 'varieties', '.', 'the', 'expression', '"', 'sa', '##uw', '##ant', '##z', '##t', '"', 'means', 'pork', 'belly', 'and', 'refers', 'also', 'to', 'un', '##ti', '##dy', 'or', 'un', '##ru', '##ly', 'children', 'and', 'youth', '.'], ['ski', '##ffle', 'is', 'a', 'music', 'genre', 'with', 'jazz', ',', 'blues', ',', 'folk', 'and', 'american', 'folk

In [38]:
# compare sequences: q - 1 q - 2 1 - 2
def compare_seqs(s1, s2):
    r = []
    for token in s1:
        if token in s2:
            r.append(token)
        else:
            r.append('-')
    return ' '.join(r)


def calculate_overlap(s1, s2):
    r = set(s1) & set(s2)
#     for token in s1:
#         if token in s2:
#             r.append(token)
    return len(r)

print(question)
print(supporting[0])
print(supporting[1])

print(compare_seqs(p1, q), '\n')  # 1st passage retrieval
print(compare_seqs(p2, q), '\n')  # 2nd passage retrieval
print(compare_seqs(p2, p1), '\n')  # bridge
print(compare_seqs(p2, q + p1))    # model

print(calculate_overlap(p1, q))  # 1st passage retrieval
print(calculate_overlap(p2, q))  # 2nd passage retrieval
print(calculate_overlap(p2, p1))  # bridge
print(calculate_overlap(p2, q + p1))    # model

Where did the form of music played by Die Rhöner Säuwäntzt originate?
Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.
Skiffle is a music genre with jazz, blues, folk and American folk influences, usually using a combination of manufactured and homemade or improvised instruments.  Originating as a term in the United States in the first half of the 20th century, it became popular again in the UK in the 1950s, where it was associated with artists such as Lonnie Donegan, The Vipers Skiffle Group, Ken Colyer and Chas McDevitt.  Skiffle played a major part in beginning the careers of later eminent jazz, pop, blues, folk and rock musicians and has been seen as a critical step

In [49]:
# check negative samples
for passage in passages.values():
    if passage not in supporting:
        words = tokenizer.tokenize(passage)
#         print(passage)  # negative sample
#         print(set(q) & set(words))
#         print(set(p1) & set(words))
        
        # compare positive and negative sequences
        
        # positive
#         print(compare_seqs(p2, q), '\n')  # 2nd passage retrieval
#         print(compare_seqs(p2, p1), '\n')  # bridge
#         print(compare_seqs(p2, q + p1))    # model
        
        
#         print(compare_seqs(words, q), '\n')  # 2nd passage retrieval
#         print(compare_seqs(words, p1), '\n')  # bridge
#         print(compare_seqs(words, q + p1))    # model
        
        print(compare_seqs(p1, p2))    # model
        print(compare_seqs(p1, words))    # model
        
        print(compare_seqs(p2, p1))    # model
        print(compare_seqs(words, p1))    # model
        
#         print(calculate_overlap(p1, p2))    # model
#         print(calculate_overlap(p1, words))    # model

        
        print(calculate_overlap(words, q))  # 2nd passage retrieval
        print(calculate_overlap(words, p1))  # bridge
        print(calculate_overlap(words, q + p1))    # model
        
#         print(calculate_overlap(p1, q+words))  # 2nd passage retrieval

        break

- - - - - - - - - a ski ##ffle - blues - - - - - - - - - in - - , - . the - - - - of - - - , - - and - - - - - ski ##ffle - blues with - - - - - - - and - - - - - . the - - - - - - - - - - - and - - to - - - or - - - - and - .
- - - - - - - - are a - - - - - from - - - - - - - in - ##n , - . the - - - - of - - ##a , - - and - - - - - - - - - with - - on - - - - and other - - - - . the - " - - - - - " - - - and - also to - - - - - - - - and - .
ski ##ffle - a - - with - , blues , - and - - - , - - a - of - and - or - - . - - a - in the - - in the - - of the - - , - - - - in the - in the - , - - - - with - - - - - - - - , the - - ski ##ffle - , - - - and - - - - - . ski ##ffle - a - - in - the - of - - - , - , blues , - and - - and - - - - a - - - to the - - - - , blues - and - - of the - - - - .
- - ##n - - - a - of - - - - - - - with - and - - on a " - - - " , a - of - - - . a - - ##n - - to - - - - - - - - - - a - - - the - of a - - - - - the - - - - in - to - - , - - - to - - - , - ,

In [45]:
# roll back

print(question)

# reconstruct p1 q+p2
print(compare_seqs(p1, q+p2), '\n')  # 1st passage retrieval
# print(compare_seqs(p1, p2), '\n')  # bridge
print(calculate_overlap(p1, q+p2))  # 2nd passage retrieval



print(supporting[0])
print(supporting[1])


Where did the form of music played by Die Rhöner Säuwäntzt originate?
die rhone ##r sa ##uw ##ant ##z ##t - a ski ##ffle - blues - - - - - - - - - in - - , - . the - - - - of - - - , - - and - - - - - ski ##ffle - blues with - - - - - - - and - - - - - . the - - sa ##uw ##ant ##z ##t - - - - and - - to - - - or - - - - and - . 

21
Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.
Skiffle is a music genre with jazz, blues, folk and American folk influences, usually using a combination of manufactured and homemade or improvised instruments.  Originating as a term in the United States in the first half of the 20th century, it became popular again in the UK in the 1950s, wh

In [46]:
# compare combinations
print(compare_seqs(p2, q + p1))    # model
print(compare_seqs(p1, q + p2))    # model




ski ##ffle - a music - with - , blues , - and - - - , - - a - of - and - or - - . - - a - in the - - in the - - of the - - , - - - - in the - in the - , where - - - with - - - - - - - - , the - - ski ##ffle - , - - - and - - - - - . ski ##ffle played a - - in - the - of - - - , - , blues , - and - - and - - - - a - - - to the - - - - , blues - and - - of the - - music - .
die rhone ##r sa ##uw ##ant ##z ##t - a ski ##ffle - blues - - - - - - - - - in - - , - . the - - - - of - - - , - - and - - - - - ski ##ffle - blues with - - - - - - - and - - - - - . the - - sa ##uw ##ant ##z ##t - - - - and - - to - - - or - - - - and - .
